In [ ]:
import pandas as pd

url = 'https://raw.githubusercontent.com/Amex2B/AmexSensitiveData/refs/heads/main/data/maker_day_stella_mini_df.csv'
df = pd.read_csv(url)
df

,Unnamed: 0.1,Unnamed: 0,unmasked_text,masked_text,privacy_mask
0,0,200,"Hello Mr. Juston Keebler, This is to remind yo...","Hello [PREFIX_1] [FIRSTNAME_1] [LASTNAME_1], T...","{'[FIRSTNAME_1]': 'Juston', '[LASTNAME_1]': 'K..."
1,1,201,"Dear Principal Interactions Executive, we've p...","Dear [JOBTITLE_1], we've partnered with an NGO...",{'[JOBTITLE_1]': 'Principal Interactions Execu...
2,2,202,"Esteemed Judge Schuppe, I write seeking your v...","Esteemed [JOBTITLE_1] [LASTNAME_1], I write se...","{'[LASTNAME_1]': 'Schuppe', '[COUNTY_1]': 'Dou..."
3,3,203,Parent Volunteers required for craft supplies ...,Parent Volunteers required for craft supplies ...,"{'[ZIPCODE_1]': '91724-4349', '[ORGANIZATION_1..."
4,4,204,"In our upcoming ethics committee meeting, we n...","In our upcoming ethics committee meeting, we n...","{'[SSN_1]': '756.6471.8923', '[PHONEIMEI_1]': ..."
...,...,...,...,...,...
195,195,1194,"Dear Mr. Powlowski (Jacky), Regarding your rec...","Dear [PREFIX_1] [LASTNAME_1] ([FIRSTNAME_1]), ...","{'[ZIPCODE_1]': '41821-3166', '[EMAIL_1]': 'De..."
196,196,1195,"I'm seeking therapy. I live in 41821-3166, my ...",I'm seeking [MEDICAL_1]. I live in [ZIPCODE_1]...,"{'[GENDER_1]': 'Cis woman', '[LASTNAME_1]': 'S..."
197,197,1196,"As Cis woman, Mrs. Smitham, you should be awar...","As [GENDER_1], [PREFIX_1] [LASTNAME_1], you sh...","{'[MASKEDNUMBER_1]': '7819906195430558', '[FIR..."
198,198,1197,Booking confirmation for the educational leade...,Booking confirmation for the educational leade...,"{'[JOBTYPE_1]': 'Architect', '[PHONENUMBER_1]'..."


In [ ]:
import random
import ast

# gets all the labels for use later
labels = []
for i in range(20):
    random_integer = random.randint(0, 19) # change to length of df but right now, keep it at 19
    labels.append(df['privacy_mask'][random_integer])

all_labels_in_20 = []
for dict in labels:
    n_dict = ast.literal_eval(dict) # loaded from a file and hasn't been converted, this solved the issue
    all_labels_in_20.append(n_dict.keys()) # gets the values such as firstname_1, lastname_1

# finding only the unique ones since there are redundant ones
unique_keys = set()

for keys in all_labels_in_20:
    for key in keys:
        result = key.strip('[]').split('_')[0]
        unique_keys.add(result)


unique_keys_list = list(unique_keys)

'''
for val in unique_keys_list:
    print(str(val))
    '''
unique_keys_list.sort()
unique_keys_list

['ACCOUNTNAME',
 'BUILDINGNUMBER',
 'CITY',
 'DATE',
 'DOB',
 'EMAIL',
 'FIRSTNAME',
 'JOBAREA',
 'JOBTITLE',
 'LASTNAME',
 'MEDICAL',
 'ORGANIZATION',
 'PHONENUMBER',
 'SEX',
 'STATE',
 'STREET',
 'TIME',
 'USERNAME',
 'ZIPCODE']

In [ ]:
#all import statements go here
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
!pip install evaluate
import evaluate
!pip install torch
import torch
from datasets import load_dataset
from sklearn.model_selection import train_test_split

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("lakshyakh93/deberta_finetuned_pii")

# taking only the important values from the privacy mask column
df['ml_label'] = ""

# iterating through the dataframe to put it into the privacy_mask column
for idx, mask in enumerate(df['privacy_mask']):
    mask_dict = ast.literal_eval(mask)
    vals = mask_dict.values()
    df.at[idx, "ml_label"] = list(vals)

'''
count = 0
for dict in df['privacy_mask']:
    val = ast.literal_eval(dict)
    position = df[df['privacy_mask'] == dict].index[0]
    print(position)
    list = []
    for num in val.values():
        list.append(num)
    print(list)
    df.loc[position, "ml_label"] = list #df['ml_label'][position] = list
'''

df

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

,Unnamed: 0.1,Unnamed: 0,unmasked_text,masked_text,privacy_mask,ml_label
0,0,200,"Hello Mr. Juston Keebler, This is to remind yo...","Hello [PREFIX_1] [FIRSTNAME_1] [LASTNAME_1], T...","{'[FIRSTNAME_1]': 'Juston', '[LASTNAME_1]': 'K...","[Juston, Keebler, cognitive therapy appointmen..."
1,1,201,"Dear Principal Interactions Executive, we've p...","Dear [JOBTITLE_1], we've partnered with an NGO...",{'[JOBTITLE_1]': 'Principal Interactions Execu...,"[Principal Interactions Executive, health camp..."
2,2,202,"Esteemed Judge Schuppe, I write seeking your v...","Esteemed [JOBTITLE_1] [LASTNAME_1], I write se...","{'[LASTNAME_1]': 'Schuppe', '[COUNTY_1]': 'Dou...","[Schuppe, Douglas County, (89)-7737 9842]"
3,3,203,Parent Volunteers required for craft supplies ...,Parent Volunteers required for craft supplies ...,"{'[ZIPCODE_1]': '91724-4349', '[ORGANIZATION_1...","[91724-4349, Teacher’s Appreciation Day, +77.5..."
4,4,204,"In our upcoming ethics committee meeting, we n...","In our upcoming ethics committee meeting, we n...","{'[SSN_1]': '756.6471.8923', '[PHONEIMEI_1]': ...","[756.6471.8923, 85-022597-998145-7, 016-314655..."
...,...,...,...,...,...,...
195,195,1194,"Dear Mr. Powlowski (Jacky), Regarding your rec...","Dear [PREFIX_1] [LASTNAME_1] ([FIRSTNAME_1]), ...","{'[ZIPCODE_1]': '41821-3166', '[EMAIL_1]': 'De...","[41821-3166, Devyn34@yahoo.com, (24)-0683.7884]"
196,196,1195,"I'm seeking therapy. I live in 41821-3166, my ...",I'm seeking [MEDICAL_1]. I live in [ZIPCODE_1]...,"{'[GENDER_1]': 'Cis woman', '[LASTNAME_1]': 'S...","[Cis woman, Smitham, 07-9586-0749, Penelope_St..."
197,197,1196,"As Cis woman, Mrs. Smitham, you should be awar...","As [GENDER_1], [PREFIX_1] [LASTNAME_1], you sh...","{'[MASKEDNUMBER_1]': '7819906195430558', '[FIR...","[7819906195430558, Damon, +611-006-618.7837]"
198,198,1197,Booking confirmation for the educational leade...,Booking confirmation for the educational leade...,"{'[JOBTYPE_1]': 'Architect', '[PHONENUMBER_1]'...","[Architect, 0414.57824332]"


In [ ]:
# splitting the dataset

X = df['ml_label']
y = df['unmasked_text']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train

,ml_label
86,"[3/23, 3 PM, Dr., Edyth, Drew, Franecki, 050.0..."
98,"[Miles, Ernser - Becker, (95) 2600-9158, Ellio..."
89,"[Transgender female, Mozilla/5.0 (Macintosh; U..."
152,"[Jayce, 034-9559-5356]"
174,"[Female to male transsexual man, Kaya, 01 49-5..."
...,...
160,"[Mr., (83).2133.2684]"
23,"[DE67040994470010410096, 7AyOICi2RTyH, 042-554..."
36,"[82826, Southwest, 05.95.65.36-80, Male, 1941-..."
170,"[8240, Griffin Squares, Rodriguezland, Ms., Wi..."


In [ ]:
# Install dependencies
!pip install transformers datasets peft accelerate

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
)
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

# Load dataset
dataset = load_dataset("conll2003")  # Replace with your dataset

# Load tokenizer with add_prefix_space=True for token alignment
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base", add_prefix_space=True)

# Tokenization and label alignment function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        max_length=128,  # Specify a maximum sequence length
        padding="max_length",  # Pad to max length for consistent tensor size
        is_split_into_words=True,
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)  # Ignore special tokens
            else:
                aligned_labels.append(label[word_id])
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply tokenization to the dataset
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

# Load DeBERTa model for token classification
model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=len(dataset["train"].features["ner_tags"].feature.names),  # Adjust based on the dataset
)

# LoRA Configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["output.dense"],  # Adjust based on inspected module names
    lora_dropout=0.1,
    bias="none",
    task_type="TOKEN_CLS",  # Specify the task type
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",  # Use "epoch" for evaluation after each epoch
    learning_rate=5e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    fp16=True,  # Enable mixed precision
    report_to="none",  # Disable reporting to external tools
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-6-409ff4e5a94d>:82: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.170100,0.169163
2,0.086000,0.119808
3,0.109100,0.108830


TrainOutput(global_step=2634, training_loss=0.2151745170830051, metrics={'train_runtime': 539.6508, 'train_samples_per_second': 78.056, 'train_steps_per_second': 4.881, 'total_flos': 3243386673123840.0, 'train_loss': 0.2151745170830051, 'epoch': 3.0})

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the fine-tuned model and tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available, otherwise CPU
model = AutoModelForTokenClassification.from_pretrained(
    "microsoft/deberta-base",
    num_labels=len(dataset["train"].features["ner_tags"].feature.names),  # Adjust based on the dataset
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base", add_prefix_space=True)

# Define label mapping (replace with your dataset-specific label mappings)
label_map = {i: label for i, label in enumerate(dataset["train"].features["ner_tags"].feature.names)}

# Function for prediction
def predict(text, model, tokenizer, label_map, device):
    # Tokenize input text
    tokenized_input = tokenizer(
        text.split(),  # Assume the input is pre-tokenized into words
        return_tensors="pt",
        truncation=True,
        max_length=128,
        padding="max_length",
        is_split_into_words=True,
    ).to(device)  # Ensure tokens are on the same device as the model

    # Make predictions
    with torch.no_grad():
        outputs = model(**tokenized_input)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=2)  # Get predicted label IDs

    # Decode predictions
    tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
    predicted_labels = [label_map[label_id.item()] for label_id in predictions[0]]

    # Pair tokens with their predicted labels
    token_label_pairs = [
        (token, label) for token, label in zip(tokens, predicted_labels)
        if token not in tokenizer.all_special_tokens  # Ignore special tokens
    ]

    return token_label_pairs

# Example text for prediction
example_text = "John Doe lives in New York and works at Google."

# Predict
predictions = predict(example_text, model, tokenizer, label_map, device)
print("Predictions:")
for token, label in predictions:
    print(f"{token}: {label}")


Some weights of DebertaForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)